# Explanations: Identifying Outliers & Biased Features

By identifying features that heavily influence the data, we can identify which features we should look at to explain unexpected query answers. We use the Awards dataset referenced in our paper to explain the following query, which retrieves the 10 Universities that have received the most award money in the area of Computer Science for 2017:
```sql
SELECT B.instName, sum(A.amount) AS totalAward
FROM Award AS A 
INNER JOIN Institution AS B ON A.aid = B.aid
WHERE A.dir = 'CISE' and A.year = 2017
GROUP BY B.instName
ORDER BY totalAward DESC
LIMIT 10
```
Dataset Link: https://www.nsf.gov/awardsearch/download.jsp

### Import & Connect to MLDB
`Note: To execute the code make sure you have verified your account using the email sent by MLDB when you create an account.`

First we need to import the MLDB Python library and establish a connection:

In [1]:
import pymldb
mldb = pymldb.Connection()

## Importing the data
The datasets are available from our Github repository. We parsed the XML data provided by NFS to generate the tables. Below are the tables and their attributes (bold attributes denote keys):

**Award**(**_aid_**, amount, title, year, startdate, enddate, dir, div)

**Institution**(**_aid_**, **_instName_**, address)

**Investigator**(**_aid_**, **_name_**, **_email_**)

In [2]:
print mldb.put('/v1/procedures/_', {
    'type': 'import.text',
    'params': {
        'dataFileUrl':
            'https://raw.githubusercontent.com/Mdevlin4/CMSC724/master/Award.csv',
        'outputDataset': 'Award',
        'delimiter': ','
        }
    })
print mldb.put('/v1/procedures/_', {
    'type': 'import.text',
    'params': {
        'dataFileUrl':
            'https://raw.githubusercontent.com/Mdevlin4/CMSC724/master/Institution.csv',
        'outputDataset': 'Institution',
        'delimiter': ','
        }
    })
print mldb.put('/v1/procedures/_', {
    'type': 'import.text',
    'params': {
        'dataFileUrl':
            'https://raw.githubusercontent.com/Mdevlin4/CMSC724/master/Investigator.csv',
        'outputDataset': 'Investigator',
        'delimiter': ','
        }
    })

<Response [201]>
<Response [201]>
<Response [201]>


### Executing the Example Query 
The query below calculates the total amount of award money in the area of Computer Science (`CISE`) for the year `2017`. The answer is unexpected because high-ranking CS schools, like Carnegie Mellon are not in the Top 5. We are going to find explanations for this query, based on the award money and its other attributes.

In [3]:
mldb.query("""
SELECT B.instName, sum(A.amount) AS totalAward
FROM Award AS A 
INNER JOIN Institution AS B ON A.aid = B.aid
WHERE A.dir = 'CISE' and A.year = 2017
GROUP BY B.instName
ORDER BY totalAward DESC
LIMIT 10
""")

,B.instName,totalAward
_rowName,,
"""[""""Clemson University""""]""",Clemson University,4425039
"""[""""University of North Carolina at Chapel Hill""""]""",University of North Carolina at Chapel Hill,3619587
"""[""""SUNY at Buffalo""""]""",SUNY at Buffalo,3162942
"""[""""University of Wisconsin-Madison""""]""",University of Wisconsin-Madison,3102391
"""[""""University of Colorado at Boulder""""]""",University of Colorado at Boulder,3024814
"""[""""Cornell University""""]""",Cornell University,2861738
"""[""""University of Illinois at Urbana-Champaign""""]""",University of Illinois at Urbana-Champaign,2838857
"""[""""Arizona State University""""]""",Arizona State University,2744283
"""[""""Carnegie-Mellon University""""]""",Carnegie-Mellon University,2661297


### Labeling Elements based on Attributes
In our example query, we are retrieving the 10 Institutions with the most Total Award Money. So to generate training data for our machine learning model, we will label the data based on its "amount" attribute. The following query assigns a label of `'1'` to Awards with an amount greater than the average award amount, and `'0'` otherwise. This will allow us to train our model based on the Awards that have the most impact on the total amount.

In [4]:
mldb.query("""
SELECT Award.* AS *, Award.amount > amtavg AS aboveAvg 
FROM Award 
INNER JOIN (
    SELECT avg(amount) AS amtavg
    FROM Award
)
LIMIT 10
""")

,aboveAvg,aid,amount,dir,div,enddate,startdate,title,year
_rowName,,,,,,,,,
[2]-[[]],0,1600011,122453,MPS,MS,06/30/2019,07/01/2016,"Non-Archimedean Techniques in Analysis, Dynami...",2016
[3]-[[]],1,1600012,1996139,GEO,OS,08/31/2019,09/01/2016,Coastal SEES: Enhancing sustainability in coas...,2016
[4]-[[]],0,1600014,10500,MPS,MS,02/28/2017,03/01/2016,Conference: Evolution Equations on Singular Sp...,2016
[5]-[[]],0,1600016,83117,ENG,CBETS,10/31/2016,11/01/2015,Rapid proposal: Fires and floods: Acquisition ...,2015
[6]-[[]],0,1600017,50000,ENG,IIP,09/30/2016,10/15/2015,I-Corps: A Tissue-engineered Nipple-Areolar Co...,2015
[7]-[[]],0,1600018,185436,GEO,AGS,05/31/2019,06/15/2016,Collaborative Research: P2C2--Ultra-High-Resol...,2016
[8]-[[]],0,1600023,180000,MPS,MS,07/31/2019,08/01/2016,Linear Partial Differential Equations on Singu...,2016
[9]-[[]],0,1600024,130476,MPS,MS,05/31/2019,06/01/2016,The Regularity of Cauchy-Riemann Mappings and ...,2016
[10]-[[]],0,1600028,73000,MPS,MS,06/30/2020,07/01/2016,Long Term Regularity of Solutions of Fluid Models,2016


### Training a Model using Award Amount
We divide our dataset into two sets: one set for training our model and one set for testing our model. We randomly select 75% of the dataset to use for training our model, keeping the other 25% for testing.

In [5]:
print mldb.put('/v1/procedures/_', {
    'type': 'classifier.train',
    'params': {
        'trainingData': """
            SELECT {* EXCLUDING (amount, aboveAvg)} AS features,
                   aboveAvg AS label FROM (
                SELECT Award.* AS *, Award.amount > amtavg AS aboveAvg FROM Award 
                INNER JOIN (
                    SELECT avg(amount) AS amtavg
                    FROM Award
                )
            ) WHERE rowHash() % 4 != 0
            """,
        'modelFileUrl': 'file://award_model.cls',
        'algorithm': 'bbdt',
        'functionName': 'score',
        'mode': 'boolean'
        }
    })

<Response [201]>


The above code creates a classifier named "score" which we use on examples from our training set to determine which attributes are most influential in the example query. The higher the score, the more likely the feature is relevant. To evaluate our classifier, we can run it on our test set (note the rowHash() % 4 != 0 vs rowHash() % 4 == 0), as shown below:

In [6]:
mldb.query("""
SELECT score({features: {* EXCLUDING (amount, aboveAvg)}}) AS *
FROM (
    SELECT Award.* AS *, Award.amount > amtavg AS aboveAvg 
    FROM Award INNER JOIN (
        SELECT avg(amount) AS amtavg
        FROM Award
    )
)
WHERE rowHash() % 4 = 0
LIMIT 10
""")

,score
_rowName,
[2]-[[]],-2.032114
[5]-[[]],-1.057065
[7]-[[]],-0.372427
[8]-[[]],-2.045538
[11]-[[]],-1.813298
[16]-[[]],0.089045
[18]-[[]],-1.510057
[29]-[[]],0.235213
[31]-[[]],-2.188061


We can evaluate how well our classifier performs by evaluating it on our testing dataset:

In [7]:
mldb.put('/v1/procedures/_', {
    'type': 'classifier.test',
    'params': {
        'testingData': """
            SELECT score: score({features: {* EXCLUDING (amount,aboveAvg)}})[score], label: aboveAvg
            FROM (
                SELECT Award.* AS *, Award.amount > amtavg AS aboveAvg FROM Award 
                INNER JOIN (
                    SELECT avg(amount) AS amtavg
                    FROM Award
                )
            ) 
            WHERE rowHash() % 4 = 0
            """,
        'outputDataset': 'award_test',
        'mode': 'boolean'
        }
    })

<Response [201]>

From the statistics above, we can see our accuracy (AUC) is about 84.6%, which is not too bad! We can use various techniques to improve this score, such as removing the biased term and training a new classifier. This allows us to find multi-variable correlations, especially when one biased term is significantly more influential than the other attributes. We can view "explanations" of our classifier to get a deeper understanding of what it is doing:

In [8]:
print mldb.put('/v1/functions/explain', {
    'type': 'classifier.explain',
    'params': {
        'modelFileUrl': 'file://award_model.cls'
        }
    })

<Response [201]>


### Example of "Explaining" Every Single Example (how much each feature influences the final score)

In [9]:
mldb.query("""
SELECT explain({features: {* EXCLUDING (amount, aboveAvg)}, label: aboveAvg}) AS *
FROM (
    SELECT Award.* AS *, Award.amount > amtavg AS aboveAvg FROM Award 
    INNER JOIN (
        SELECT avg(amount) AS amtavg
        FROM Award
    )
)
WHERE rowHash() % 4 = 0
LIMIT 10
""")

,bias,explanation.aid,explanation.dir,explanation.div,explanation.enddate,explanation.startdate,explanation.title,explanation.year
_rowName,,,,,,,,
[2]-[[]],-0.045136,0.385569,0.146593,1.461342,-0.017115,0.043063,0.065828,-0.008029
[5]-[[]],-0.045136,0.366645,0.189373,0.111751,0.346428,0.026540,0.019873,0.041591
[7]-[[]],-0.045136,0.378865,-0.058162,0.111751,-0.057476,0.030371,0.020243,-0.008029
[8]-[[]],-0.045136,0.631421,0.146593,1.461342,-0.249543,0.043063,0.065828,-0.008029
[11]-[[]],-0.045136,0.385569,0.134114,1.548056,-0.251980,-0.013612,0.065144,-0.008856
[16]-[[]],-0.045136,0.378865,0.209173,0.111751,-0.760425,0.030371,-0.005615,-0.008029
[18]-[[]],-0.045136,0.362963,0.153119,0.053313,0.907648,0.030371,0.055808,-0.008029
[29]-[[]],0.045136,-0.378865,-0.083463,-0.111751,0.522608,0.236919,-0.003400,0.008029
[31]-[[]],-0.045136,0.385569,0.114443,1.461342,-0.017115,0.224374,0.072614,-0.008029


### Example of Aggregating Explanation Scores by Attribute

In [10]:
mldb.query("""
SELECT *
FROM transpose((
    SELECT avg({explain({features: {* EXCLUDING (amount,aboveAvg)}, label: aboveAvg})[explanation] as *}) AS *
    NAMED 'explanation'
    FROM (
        SELECT Award.* AS *, Award.amount > amtavg AS aboveAvg FROM Award 
        INNER JOIN (
            SELECT avg(amount) AS amtavg
            FROM Award
        )
    )
    WHERE rowHash() % 4 = 0
))
ORDER BY abs(explanation) DESC
""")

,explanation
_rowName,
enddate,0.309895
div,0.226351
dir,0.097955
aid,0.082949
startdate,0.046689
title,0.037705
year,0.007981


By aggregating the explanation scores by attribute, we notice three stand out: `enddate`, `div`, and `dir`. The attributes `div` and `dir` reprsent the Division and Directorate respectively. Since Divisions fall under Directorates, its not surprising to see both values up there since they are related to each other. Since our dataset is only from 2016-2017, `enddate` is likely related to the duration of the award, and it makes sense that longer durations would have higher amounts.

Note that since `aid` is the unique primary key for `Award`, we can infer that `aid` is not a notable attribute for identifying correlated or biased terms (meaning we can initally remove it from the set of attributes, shown below). 

## Retraining Without the Biased Features: `enddate` and `startdate`
We can look at the effects of removing `enddate` and `startdate` by adding it to the excluded columns so that it is not used by the model. We will also remove `aid` based on the intuition described above. This allows us to identify other potential outliers and gain a better understanding of our data.

In [11]:
print mldb.put('/v1/procedures/_', {
    'type': 'classifier.train',
    'params': {
        'trainingData': """
        
            SELECT {* EXCLUDING (amount, aboveAvg, aid, enddate,startdate)} AS features,
                   aboveAvg AS label
            FROM (
                SELECT Award.* AS *, Award.amount > amtavg AS aboveAvg FROM Award 
                INNER JOIN (
                    SELECT avg(amount) AS amtavg
                    FROM Award
                )
            )
            WHERE rowHash() % 4 != 0
            """,
        'modelFileUrl': 'file://award_model.cls',
        'algorithm': 'bbdt',
        'functionName': 'score',
        'mode': 'boolean'
        }
    })

<Response [201]>


In [12]:
mldb.put('/v1/procedures/_', {
    'type': 'classifier.test',
    'params': {
        'testingData': """
            SELECT score: score({features: {* EXCLUDING (amount, aboveAvg, aid, enddate, startdate)}})[score], label: aboveAvg
            FROM (
                SELECT Award.* AS *, Award.amount > amtavg AS aboveAvg FROM Award 
                INNER JOIN Investigator ON Investigator.aid = Award.aid
                INNER JOIN (
                    SELECT avg(amount) AS amtavg
                    FROM Award
                )
            )
            WHERE rowHash() % 4 = 0
            """,
        'outputDataset': 'award_test',
        'mode': 'boolean'
        }
    })

<Response [201]>

We can see that our accuracy (AUC) has taken a pretty serious hit, dropping from 85% to 74%. This is because we eliminated two attributes from our datasets that are now no longer used in classification (`enddate`, `startdate`, and `aid`).

This allows us to better understand the next most influential attributes as well as correlated variables that were difficult to observe due to biased features. The effect of removing these terms is shown below:

In [13]:
print mldb.put('/v1/functions/explain', {
    'type': 'classifier.explain',
    'params': {
        'modelFileUrl': 'file://award_model.cls'
        }
    })

<Response [201]>


In [14]:
mldb.query("""
SELECT *
FROM transpose((
    SELECT avg({explain({features: {* EXCLUDING (amount, aboveAvg, aid, enddate,startdate)}, label: aboveAvg})[explanation] as *}) AS *
    NAMED 'explanation'
    FROM (
        SELECT Award.* AS *, Award.amount > amtavg AS aboveAvg FROM Award 
        INNER JOIN (
            SELECT avg(amount) AS amtavg
            FROM Award
        )
    )
    WHERE rowHash() % 4 = 0
))
ORDER BY abs(explanation) DESC
""")

,explanation
_rowName,
div,0.237729
dir,0.089102
year,0.049145
title,0.033307


After removing the biased attributes, we see `div` and `dir` are the two most biased attributes left. Since `div` is correlated to the total amount, it makes sense that `dir` is as well, because directorates and divisions are related by a hierarchal model (so if `div` influences the result heavily, it makes sense that `dir` would as well).


We can also look at the instituion's total amount for each Division under `'CISE'`:

In [23]:
mldb.query("""
SELECT B.instName, A.div, sum(A.amount) AS totalAward
FROM Award AS A 
INNER JOIN Institution AS B ON A.aid = B.aid
WHERE A.dir = 'CISE' and A.year = 2017
GROUP BY B.instName, A.div
ORDER BY totalAward DESC
""")

,A.div,B.instName,totalAward
_rowName,,,
"""[""""University of North Carolina at Chapel Hill"""",""""OAC""""]""",OAC,University of North Carolina at Chapel Hill,3483303
"""[""""Clemson University"""",""""OAC""""]""",OAC,Clemson University,2952217
"""[""""SUNY at Buffalo"""",""""OAC""""]""",OAC,SUNY at Buffalo,2749699
"""[""""US Ignite, Inc."""",""""CNS""""]""",CNS,"US Ignite, Inc.",2655164
"""[""""Carnegie-Mellon University"""",""""CNS""""]""",CNS,Carnegie-Mellon University,1915755
"""[""""University of Missouri-Columbia"""",""""CNS""""]""",CNS,University of Missouri-Columbia,1709212
"""[""""University of Houston"""",""""CNS""""]""",CNS,University of Houston,1667549
"""[""""University of Wisconsin-Madison"""",""""IIS""""]""",IIS,University of Wisconsin-Madison,1544954
"""[""""University of Colorado at Boulder"""",""""AC""""]""",AC,University of Colorado at Boulder,1497820


## Finding Explanations Between Multiple Tables
Using the technique described above, we can identify the attributes relevant for an explanation by examining biased terms in the query. In the examples above, we only looked at attributes in Award to find biased terms, but this could miss inter-table relationships. To examine attributes from multiple tables efficiently, we can join tables on foriegn keys which will help limit the number of resulting records, and eliminates the noise generated by joining on unrelated attributes.

For example, if we want to see if the Investigator is related to the Award amount, we can join the tables on `aid` and classify each Investigator based on whether the Award amount was higher or lower than the average award amount. This is shown in the query below:

In [42]:
mldb.query("""
SELECT Invest.name AS name, Invest.email AS email, Inst.instName, A.amount > amtavg AS aboveAvg
FROM Award AS A
INNER JOIN Investigator AS Invest ON Invest.aid = A.aid
INNER JOIN Institution AS Inst ON Inst.aid = A.aid
INNER JOIN (
            SELECT avg(amount) AS amtavg
            FROM Award
            )
ORDER BY name
LIMIT 10
""")

,Inst.instName,aboveAvg,email,name
_rowName,,,,
[7524]-[7524]-[7173]-[[]],University of Southern California,0,joshwest@usc.edu,A Joshua West
[10505]-[10505]-[10150]-[[]],University of California-Santa Cruz,0,acr@ucsc.edu,A. Christina Ravelo
[4074]-[4074]-[3748]-[[]],Montclair State University,0,david.trubatch@montclair.edu,A. David Trubatch
[7343]-[7343]-[6992]-[[]],National Academy of Sciences,1,esztein@nas.edu,A. Ester Sztein
[652]-[652]-[648]-[[]],South Dakota State University,1,joshua.leffler@sdstate.edu,A. Joshua Leffler
[2602]-[2602]-[2466]-[[]],Davidson College,1,macampbell@davidson.edu,A. Malcolm Campbell
[1450]-[1450]-[1391]-[[]],Temple University,0,lyyra@temple.edu,A. Marjatta Lyyra
[1359]-[1359]-[1302]-[[]],Yale University,1,morse@sysc.eng.yale.edu,A. Morse
[7092]-[7092]-[6741]-[[]],University of Maryland College Park,0,ssh@astro.umd.edu,A. Surjalal Sharma


Below shows an example of training a classifier and running it for the above example. Note the low accuracy (under 50%) and the low explanations, meaning there is probably not a relation with award amount.

In [44]:
print mldb.put('/v1/procedures/_', {
    'type': 'classifier.train',
    'params': {
        'trainingData': """
        
            SELECT {* EXCLUDING (aboveAvg)} AS features,
                   aboveAvg AS label
            FROM (
                SELECT Invest.name AS name, Invest.email AS email, A.amount > amtavg AS aboveAvg
                FROM Award AS A
                INNER JOIN Investigator AS Invest ON Invest.aid = A.aid
                INNER JOIN Institution AS Inst ON Inst.aid = A.aid
                INNER JOIN (
                            SELECT avg(amount) AS amtavg
                            FROM Award
                            )
            )
            WHERE rowHash() % 4 != 0
            """,
        'modelFileUrl': 'file://award_model.cls',
        'algorithm': 'bbdt',
        'functionName': 'score',
        'mode': 'boolean'
        }
    })

<Response [201]>


In [45]:
mldb.put('/v1/procedures/_', {
    'type': 'classifier.test',
    'params': {
        'testingData': """
            SELECT score: score({features: {* EXCLUDING (aboveAvg)}})[score], label: aboveAvg
            FROM (
                SELECT Invest.name AS name, Invest.email AS email, A.amount > amtavg AS aboveAvg
                FROM Award AS A
                INNER JOIN Investigator AS Invest ON Invest.aid = A.aid
                INNER JOIN Institution AS Inst ON Inst.aid = A.aid
                INNER JOIN (
                            SELECT avg(amount) AS amtavg
                            FROM Award
                            )
            )
            WHERE rowHash() % 4 = 0
            """,
        'outputDataset': 'award_test',
        'mode': 'boolean'
        }
    })

<Response [201]>

In [49]:
print mldb.put('/v1/functions/explain', {
    'type': 'classifier.explain',
    'params': {
        'modelFileUrl': 'file://award_model.cls'
        }
    })
mldb.query("""
SELECT *
FROM transpose((
    SELECT avg({explain({features: {* EXCLUDING (aboveAvg)}, label: aboveAvg})[explanation] as *}) AS *
    NAMED 'explanation'
    FROM (
            SELECT Invest.name AS name, Invest.email AS email, A.amount > amtavg AS aboveAvg
            FROM Award AS A
            INNER JOIN Investigator AS Invest ON Invest.aid = A.aid
            INNER JOIN Institution AS Inst ON Inst.aid = A.aid
            INNER JOIN (
                        SELECT avg(amount) AS amtavg
                        FROM Award
                        )
        )
    WHERE rowHash() % 4 = 0
))
ORDER BY abs(explanation) DESC
""")

<Response [201]>


,explanation
_rowName,
name,0.121261
email,0.041612


## Conclusion
Using ML techniques, we were able to correctly identify correlated attributes, which are useful in explaining unexpected query answers. These techniques can be applied to incorporate automatic attribute selection into explainable database by exploiting primary & foriegn key relationships, knowledge about aggregate operators, and minimal human domain knowledge.